In [25]:
Anime.objects.count()

10315

In [26]:
Manga.objects.count()

3857

In [30]:
NB_MANGA_LIMIT = 20
NB_ANIME_LIMIT = 20
NB_USERS_LIMIT = 200

SearchIssue.objects.all().delete()
# Profile.objects.all().delete()  # Keeping some profiles would be better
Suggestion.objects.all().delete()
Recommendation.objects.all().delete()
Pairing.objects.all().delete()
SocialApp.objects.all().delete()
Session.objects.all().delete()
ContentType.objects.all().delete()
# Reference.objects.all().delete()

(0, {})

In [29]:
from collections import Counter
nb = Counter(Rating.objects.values_list('work_id', flat=True))
# Distinction between Anime and Manga will disappear with PR #153
if NB_ANIME_LIMIT:
    work_ids = list(Anime.objects.values_list('id', flat=True))
    work_ids.sort(key=lambda work_id: -nb[work_id])
    Anime.objects.exclude(id__in=work_ids[:NB_ANIME_LIMIT]).delete()
if NB_MANGA_LIMIT:
    work_ids = list(Manga.objects.values_list('id', flat=True))
    work_ids.sort(key=lambda work_id: -nb[work_id])
    Manga.objects.exclude(id__in=work_ids[:NB_MANGA_LIMIT]).delete()

In [33]:
if NB_ANIME_LIMIT or NB_MANGA_LIMIT:
    bundle = []
    for artist_ids in Work.objects.values_list('artists'):
        bundle.extend(artist_ids)
    kept_artist_ids = list(set([x for x in bundle if x is not None]))
    Artist.objects.exclude(id__in=kept_artist_ids).delete()

In [34]:
Artist.objects.count()

48

In [35]:
Work.objects.count()

40

In [36]:
Rating.objects.count()

3163

In [37]:
max_user_id = max(User.objects.values_list('id', flat=True))

In [38]:
import random
chosen = User.objects.order_by('?')[:NB_USERS_LIMIT]
kept_ids = chosen.values_list('id', flat=True)
User.objects.exclude(id__in=kept_ids).delete()

(0, {})

In [39]:
User.objects.count()

200

In [40]:
from django.utils import timezone
new_ids = random.sample(range(max_user_id + 1, max_user_id + 201), 200)
for user, new_id in zip(User.objects.all(), new_ids):
    old_id = user.id
    user.pk = new_id
    user.username = str(new_id)
    user.is_superuser = False
    user.set_password('mangaki')
    user.email = '%d@mangaki.fr' % new_id
    user.date_joined = timezone.now()
    user.last_login = timezone.now()
    user.save()
    old_user = User.objects.get(id=old_id)
    old_user.rating_set.update(user=user)
    # old_user.profile.user = user
    old_user.delete()

In [41]:
User.objects.all()

[<User: 2156>, <User: 2261>, <User: 2221>, <User: 2202>, <User: 2120>, <User: 2200>, <User: 2226>, <User: 2159>, <User: 2192>, <User: 2121>, <User: 2186>, <User: 2242>, <User: 2203>, <User: 2180>, <User: 2241>, <User: 2157>, <User: 2298>, <User: 2245>, <User: 2154>, <User: 2113>, '...(remaining elements truncated)...']

Now, please make ``./mangaki/manage.py dumpdata > fixtures/seed_data.json``

In [42]:
from collections import Counter
import json
j = json.load(open('../fixtures/seed_data.json'))
Counter(entry['model'] for entry in j)

Counter({'auth.group': 1,
         'auth.user': 200,
         'irl.event': 38,
         'irl.location': 2,
         'irl.partner': 12,
         'mangaki.anime': 20,
         'mangaki.artist': 48,
         'mangaki.category': 3,
         'mangaki.editor': 17,
         'mangaki.genre': 34,
         'mangaki.manga': 20,
         'mangaki.page': 4,
         'mangaki.rating': 3163,
         'mangaki.reference': 30,
         'mangaki.role': 5,
         'mangaki.staff': 100,
         'mangaki.studio': 50,
         'mangaki.top': 3,
         'mangaki.work': 40,
         'sites.site': 1})